In [ ]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "rag-app"

In [5]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3.2", model_provider="ollama", base_url ="http://localhost:11434")

In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2", base_url="http://localhost:11434")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="rag-pipeline",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [10]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()